# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('orders.zip')
orders.head()

Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75       17850  United Kingdom         22.00  
3       3.39       17850  United Kingdom         20.34  
4       3.39       17850  United Kingdom         20.34

In [3]:
orders.shape

(397924, 14)

In [4]:

orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# your code here
grouped_orders = orders.groupby('CustomerID')
amount_per_customer = grouped_orders[['amount_spent']].agg(sum)
amount_per_customer


amount_spent
CustomerID              
12346           77183.60
12347            4310.00
12348            1797.24
12349            1757.55
12350             334.40
...                  ...
18280             180.60
18281              80.82
18282             178.05
18283            2094.88
18287            1837.28

[4339 rows x 1 columns]

In [6]:
VIP_customers = amount_per_customer[amount_per_customer['amount_spent'] > np.percentile(amount_per_customer['amount_spent'], 95)]
VIP_customers

amount_spent
CustomerID              
12346           77183.60
12357            6207.67
12359            6372.58
12409           11072.67
12415          124914.53
...                  ...
18109            8052.97
18139            8438.34
18172            7561.68
18223            6484.54
18229            7276.90

[217 rows x 1 columns]

In [7]:
preferred_customers = amount_per_customer[(amount_per_customer['amount_spent'] >= np.percentile(amount_per_customer['amount_spent'], 75)) & (amount_per_customer['amount_spent'] <= np.percentile(amount_per_customer['amount_spent'], 95))]
preferred_customers

amount_spent
CustomerID              
12347            4310.00
12348            1797.24
12349            1757.55
12352            2506.04
12356            2811.43
...                  ...
18259            2338.60
18260            2643.20
18272            3078.58
18283            2094.88
18287            1837.28

[868 rows x 1 columns]

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
VIP_customers.index

Int64Index([12346, 12357, 12359, 12409, 12415, 12428, 12431, 12433, 12435,
            12451,
            ...
            17865, 17949, 18055, 18092, 18102, 18109, 18139, 18172, 18223,
            18229],
           dtype='int64', name='CustomerID', length=217)

In [9]:

condition = orders['CustomerID'].isin(VIP_customers.index)
filtered_orders =orders[condition]
filtered_orders['CustomerID'].value_counts() 
#to verify that we have selected only the VIP customers among the total customers and filtered correctly, I use value counts. 
#it shows me the same quantity as my VIP_customers (217), so I can conclude the filter worked.

17841    7847
14911    5677
14096    5111
12748    4596
14606    2700
         ... 
12798       8
16532       4
15098       3
16446       3
12346       1
Name: CustomerID, Length: 217, dtype: int64

In [10]:
filtered_orders.groupby('Country')['CustomerID'].nunique()
#nunique to identify the unique values per country: UK has by far more VIP customers than other countries.

Country
Australia            2
Belgium              1
Channel Islands      1
Cyprus               1
Denmark              1
EIRE                 2
Finland              1
France               9
Germany             10
Japan                2
Netherlands          1
Norway               1
Portugal             2
Singapore            1
Spain                2
Sweden               1
Switzerland          3
United Kingdom     177
Name: CustomerID, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [11]:
# your code here
condition = ((orders['CustomerID'].isin(VIP_customers.index)) |(orders['CustomerID'].isin(preferred_customers.index)))
filtered_orders_2 =orders[condition]
filtered_orders_2['CustomerID'].value_counts() 

17841    7847
14911    5677
14096    5111
12748    4596
14606    2700
         ... 
18087       2
12346       1
17846       1
13135       1
15195       1
Name: CustomerID, Length: 1085, dtype: int64

In [12]:
filtered_orders_2.groupby('Country')['CustomerID'].nunique()
#the country with the most VIP + preferred customer qty is still the UK

Country
Australia            4
Austria              3
Belgium             12
Canada               1
Channel Islands      4
Cyprus               4
Denmark              3
EIRE                 3
Finland              5
France              29
Germany             39
Greece               1
Iceland              1
Israel               2
Italy                5
Japan                4
Lebanon              1
Malta                1
Netherlands          1
Norway               7
Poland               1
Portugal             7
Singapore            1
Spain                9
Sweden               2
Switzerland          9
United Kingdom     932
Name: CustomerID, dtype: int64